# Lab: Gender gaps

## Source (Dataset)
Office of the National Statistics Gender Pay Gap [ONS Source](https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/annualsurveyofhoursandearningsashegenderpaygaptables)

## Explanations (from the source)
+ Gender pay gap (GPG) - calculated as the difference between average hourly earnings (excluding overtime) of men and women as a proportion of average hourly earnings (excluding overtime) of men. For example, a 4% GPG denotes that women earn 4% less, on average, than men. Conversely, a -4% GPG denotes that women earn 4% more, on average, than men.

+ Mean: a measure of the average which is derived by summing the values for a given sample, and then dividing the sum by the number of observations (i.e. jobs) in the sample. In earnings distributions, the mean can be disproportionately influenced by a relatively small number of high-paying jobs.
																						
+ Median: the value below which 50% of jobs fall. It is ONS's preferred measure of average earnings as it is less affected by a relatively small number of very high earners and the skewed distribution of earnings. It therefore gives a better indication of typical pay than the mean.
																			
### Coverage and timeliness																									
The Annual Survey of Hours and Earnings (ASHE) covers employee jobs in the United Kingdom. It does not cover the self-employed, nor does it cover employees not paid during the reference period (2023).

GPG estimates are provided for the pay period that included a specified date in April. They relate to employees on adult rates of pay, whose earnings for the survey pay period were not affected by absence. 
	
ASHE is based on a 1% sample of jobs taken from HM Revenue and Customs' Pay As You Earn (PAYE) records. Consequently, individuals with more than one job may appear in the sample more than once.																	 

## Reading the dataset


In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_profession = pd.read_excel('data/genderpaygap.xlsx', sheet_name='All')
df_profession_category = pd.read_excel('data/genderpaygap.xlsx', sheet_name='Main')
df_age = pd.read_excel('data/genderpaygap.xlsx', sheet_name='Age')
df_geography = pd.read_excel('data/genderpaygap.xlsx', sheet_name='Geography')

Let's have a look at our dataset

In [ ]:
df_profession.tail()

In [ ]:
df_profession_category.tail()

In [ ]:
df_age

In [ ]:
df_geography.tail()

If you look at the Excel data files, we see that occupations have a main and sub-category. Since we have the main category values in df_profession_category anyway, let's drop them from 'df_profession' to retain the focus on sub-categories only. We can do this based on the values in the Code column since as you can see main category professions have code values < 10 and sub-categories have values greater than 10. 

In [ ]:
indices_to_drop = df_profession[df_profession['Code'] < 10].index
df_profession.drop(indices_to_drop, inplace=True)
df_profession

### Missing values
Let's check our data 

In [ ]:
df_profession.info()
df_profession_category.info()
df_age.info()
df_geography.info()

In [ ]:
# It looks like GPGmean is read as an object (string) in df_profession dataframe. 
# GPGmean and GPGmedian are both objects in df_geography
# Let's convert the data to float64, so we can create plots later
df_profession['GPGmean'] = pd.to_numeric(df_profession['GPGmean'], errors='coerce')
df_geography['GPGmean'] = pd.to_numeric(df_geography['GPGmean'], errors='coerce')
df_geography['GPGmedian'] = pd.to_numeric(df_geography['GPGmedian'], errors='coerce')

In [ ]:
# Next, let's check for missing values
df_profession.isna().sum()
df_profession_category.isna().sum()
df_age.isna().sum()

All seems fine - let's get plotting

In [ ]:
# Let's plot the mean and median Gender Pay Gap (GPG)
df_profession.boxplot(column=['GPGmedian', 'GPGmean'])

Hmmm, there are outliers. Let's check the descriptive statistics

In [ ]:
# Let's look at the distribution of the values in the columns
df_profession.describe()

In [ ]:
# Let's try to visualise what's going on with a histogram - what type of skew do you notice?
df_profession[['GPGmedian']].plot(kind='hist', ec='black')

Hmmm, there appears to be a lone bin in our histogram. Which might be the profession or professions where women earn more than men?

In [ ]:
# Is there one profession or more professions where women earn more? Let's do some investigation through visualisation. 
import altair as alt
alt.Chart(df_profession).mark_bar().encode(
    alt.X("GPGmedian:Q", bin=True, title='GPGmedian'), 
    y=alt.Y('Description:N', sort='-x', title='Professional Category'),  
    color='Description:N',
    tooltip=['Description', 'GPGmedian']
).properties(
    width=600,
    height=400
)

This plot shows us that Community and civil enforcement occupations, skilled agricultural and related trades, and secretarial and related occupations are the ones where women earn, on average, more than men. 

If you are wondering what 'community and civil enforcement occupations' mean - then [this ONS source](https://www.ons.gov.uk/methodology/classificationsandstandards/standardoccupationalclassificationsoc/soc2020/soc2020volume1structureanddescriptionsofunitgroups) says it includes police community and parking and civil enforcement officers.

Are these occupations the ones you suspected women to earn more than men (on average)?

<span style="color: blue;">Sidenote:</span> The above visualisation is detailed, but it's busy and cluttered. How about if we try doing this on df_profession_category instead?

In [ ]:
# Is there one profession where Women earn more? Let's do some investigation. 
import altair as alt
alt.Chart(df_profession_category).mark_bar().encode(
    alt.X("GPGmedian:Q", bin=True), 
    y=alt.Y('Description:N', sort='-x'),  
    color='Description:N',
    tooltip=['Description', 'GPGmedian']
).properties(
    width=600,
    height=400
)

In this, we have lost some of the detail we had in the earlier visualisation, but we get to know that "Caring, leisure and other service occupations" is a 'main category' of occcupation where the GPG is low (but women don't earn more than men). 

<span style="color: blue;">Sidenote:</span> What does this narrative tell you about women being more likely to do multiple jobs to work around their domestic responsibilities which we spoke about in the lecture (and recordings)?

In [ ]:
# Alternative visualisation (excluding all employees category)

# In which main professional categories is the gap narrow? Let's find out!
df_professions_sorted = df_profession_category.sort_values('GPGmedian', ascending=True)

# Let's drop the row corresponding to 'All employees' because we are more interested in looking at the differences across professional categories and sub-categories here 
df_professions_sorted = df_professions_sorted[df_professions_sorted['Description'] != 'All employees']

# Let's create the bar plot
df_professions_sorted.plot.bar(x='Description', y='GPGmedian')

Let's look at age-based differences next:

In [ ]:
df_age.sort_values('age_group', ascending=True).plot.bar(x = 'age_group', y = 'GPGmedian')

<span style="color: blue;">Reflection:</span> It seems that GPG increases with age - what does this say about our dicussion during the lectures about GPG increasing for women who take time off from work for a variety of reasons compared to their male and female counterparts who do not take time out of work! What do you think might be the reasons for the minor fall in GPG at 60+?

## Geography

#### But first: 
Since moving on from our Iris and Wine datasets, the real-world datasets rarely come prepared (ready to use).

+ If you download the zip file for the latest [2023 Pay Gap ONS statistics](https://www.ons.gov.uk/file?uri=/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/annualsurveyofhoursandearningsashegenderpaygaptables/2023provisional/genderpaygap2023provisional.zip), you will notice that they have color-coded their cells based on the certainty of estimates. On the one hand, this is very good practice - being transparent about the quality of the data. On the other hand, it tells us that we need to be careful about what insights we can draw from the data.
+ If you look at the statistics for one year, you can get a glimpse of what's happening across various categories (geography, age, profession, etc.) in terms of GPG but it's a cross-sectional view. But you can collate a longitudinal view should you wish to. E.g., by downloading the zip folders across the desired years and collating the information for desired categories for multiple years. But remember that this will be a 'simplified approach' to a longitudinal view and will have limitations. Also, recollect one of the figures Cagatay showed in the earlier lectures - it's common to spend a lot of time at the start of your Data Science project just collating the necessary information. If you fancy, you can write a script to automate the data collation process!
+ We have geography information as area codes from the ONS source, but wouldn't it be nice if we are able to visualise GPG by Geography on a map of England (with [Levelling Up agenda](https://www.gov.uk/government/publications/levelling-up-the-united-kingdom) and all). That's the data hunt I went on. And found this:[UK GEO-Json](https://martinjc.github.io/UK-GeoJSON/). Now let's see what visualisation we can create with it.
  
<span style="color: blue;">Takeaway:</span> Collating data from multiple sources is a significant, valuable and legitimate part of the Data Science project journey

In [ ]:
# Getting the geospatial polygons for England
import geopandas as gpd 
import altair as alt

geo_states_england = gpd.read_file('data/Counties_and_Unitary_Authorities_May_2023_UK_BUC_-7406349609691062173.gpkg')
geo_states_england.head()

In [ ]:
print(geo_states_england.columns)

In [ ]:
# Let's drop the columns we don't need
geo_states_england = geo_states_england.drop(['CTYUA23NMW', 'BNG_E', 'BNG_N', 'GlobalID'], axis=1)

In [ ]:
# Let's check again
geo_states_england.head()

In [ ]:
# Let's create a map of England
pre_GPG_England = alt.Chart(geo_states_england, title='Map of England').mark_geoshape().encode(
    tooltip=['CTYUA23NM']
).properties(
    width=500,
    height=300
)
pre_GPG_England

What's that?!

::: callout-warning

## Map projections

Because the Earth is round, and maps are flat, geospatial data needs to be "projected". There are many types of projecting geospatial data, and all of them come with some tradeoff in terms of distorting area and/or distance (in other words, none of them are perfect). You can read more here.

Now, the geospatial dataset that we are using for this notebook was downloaded from and uses a Coordinate Reference System (CRS)
known as EPSG:27700 - OSGB36 / British National Grid. Regretfully, Altair works with a different CRS:  WGS 84 (also known as epsg:4326), and this is creating the conflict.

We have two options: either reproject our data using geopandas, or according to [Altair documentation](https://altair-viz.github.io/user_guide/data.html#projections) try using the project configuration `(type: 'identity', reflectY': True)``. It draws the geometries without applying a projection.

:::

In [ ]:
# Let's create a map of England
pre_GPG_England = alt.Chart(geo_states_england, title='Map of England').mark_geoshape().encode(
    tooltip=['CTYUA23NM']
).properties(
    width=500,
    height=300
).project(
    type='identity',
    reflectY=True
)
pre_GPG_England

In [ ]:
df_geography.info()

In [ ]:
geo_states_england

In [ ]:
df_geography

In [ ]:
# Add the data
geo_states_england = geo_states_england.merge(df_geography, left_on = 'CTYUA23CD', right_on = 'Code')

In [ ]:
# Check the merged data
geo_states_england.head(10)

In [ ]:
# Let's plot the GPG by geography now
post_GPG_England = alt.Chart(geo_states_england, title='GPG by region - England').mark_geoshape().encode(
    color='GPGmedian',
    tooltip=['Description', 'GPGmedian']
).properties(
    width=500,
    height=300
).project(
    type='identity',
    reflectY=True
)
post_GPG_England

In [ ]:
# side by side view
GPG_England = pre_GPG_England | post_GPG_England
GPG_England

<span style="color: blue;">NOTE:</span> There seems to be a blip in the rendering of the post_GPG_England map. Do you think a cleaner dataset will help overcome this issue?

How do the results in this workbook compare to the visualisation we saw during the lecture, for example, for the UK in [Information is Beautiful](https://informationisbeautiful.net/visualizations/gender-pay-gap/) But remember the earnings across the two might be for different years - do remember to check the metadata!